In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
os.chdir('/mnt/BioAdHoc/Groups/vd-ay/hichip-db-loop-calling')

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('max_colwidth', 400)

outdir = 'results/peaks/'
ref = 'hg38'
rerun = True

In [2]:
assigned_date = '03_08_24' # MM.DD.YY

# Process ChIP-seq Data

### Helper functions

In [3]:
def read_chipseq_files(file):
    peak_data = {}
    count = 0
    with open(file) as fr:
        for line in fr:
            info = line.strip().split()
            peak_info = info[0:3]
            peak_data[count] = peak_info
            count = count + 1
    return(peak_data)

In [4]:
def get_chipseq_path(samplesheet, sample_name):
    path=''
    with open(samplesheet) as fr:
        for line in fr:
            if sample_name in line:
                info = line.strip().split()
                path = info[2]
    return(path) 

In [5]:
def generate_chipseq_df(outfn='test.tsv', rerun=False):
    
    if rerun == True:
        data = []
        glob_str = 'results/hicpro/*Homo*/hic_results'

        for file in glob.glob(glob_str):
            
            # get sample information
            sample_name = file.split('/')[2]
            sample_info = []
            sample_info.append(sample_name)

            # parse MACS2 data
            fn = 'results/samplesheets/post-hicpro/2024.2.15.10.52.peaks_files_chipseq.all_batches.samplesheet.without_header.tsv'
            chip_path = get_chipseq_path(fn, sample_name)
            if os.path.exists(chip_path):
                peak_info = read_chipseq_files(chip_path)
                sample_info.append(len(peak_info))
                peak_sizes = []
                for peak in peak_info.items():
                    peak_sizes.append(int(int(peak[1][2])-int(peak[1][1])))
                sample_info.append(np.mean(peak_sizes))
            else:
                sample_info.append(-1)
                sample_info.append(-1)
            data.append(sample_info)

        # create a dataframe and save to file
        df = pd.DataFrame(data)
        df.to_csv(outfn, sep='\t', index=None)
        
    else:
        # load old data when re-run is not required
        df = pd.read_table(outfn)
        
    return df

In [6]:
# extract the chipseq_std_sample_name
def get_chipseq_std(x):
    if len(x) > 2:
        
        # sometimes a matching chipseq file could not be found, return -1
        if x[-1] == 'Could not find a matching file':
            return(-1)
        
        # else return the chipseq std sample name within the path
        else:
            return(x[-3])
    else:
        return(-1)

### Loading data

In [7]:
# load the hicpro samples
# samples = pd.read_table('results/samplesheets/hicpro/current.hicpro.samplesheet.without_header.tsv', header=None)
# columns = ['std_sample_name', 'gse_id', 'organism', 'bio_rep', 'antibody_target', 'restriction_enzyme', 'sample_name']
# samples.columns = columns

In [8]:
# load mapping between hicpro and chipseq std sample names
fn = 'results/samplesheets/post-hicpro/2024.2.1.10.52.peaks_files_chipseq.batch1.samplesheet.without_header.tsv'
batch1_df = pd.read_table(fn, names=['hicpro_std_sample_name', 'hicpro_path', 'chipseq_path'])

fn = 'results/samplesheets/post-hicpro/2024.2.1.10.52.peaks_files_chipseq.batch2.samplesheet.without_header.tsv'
batch2_df = pd.read_table(fn, names=['hicpro_std_sample_name', 'hicpro_path', 'chipseq_path'])

# concating the batches
hicpro_to_chipseq = pd.concat([batch1_df, batch2_df])

# extracting the chipseq std sample name
hicpro_to_chipseq['chipseq_std_sample_name'] = hicpro_to_chipseq.chipseq_path.str.split('/').apply(get_chipseq_std)

In [9]:
hicpro_to_chipseq.loc[hicpro_to_chipseq['chipseq_std_sample_name'] == -1].shape

(4, 4)

In [10]:
hicpro_to_chipseq.head()

,hicpro_std_sample_name,hicpro_path,chipseq_path,chipseq_std_sample_name
0,H9.GSE105028.Homo_Sapiens.CTCF.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/H9.GSE105028.Homo_Sapiens.CTCF.b1/hic_results/data/H9.GSE105028.Homo_Sapiens.CTCF.b1/H9.GSE105028.Homo_Sapiens.CTCF.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/H9.GSE105028.Homo_Sapiens.CTCF.b1/MACS2_Ext_No_Control/H9.GSE105028.Homo_Sapiens.CTCF.b1.macs2_peaks.narrowPeak_Q0.01filt,H9.GSE105028.Homo_Sapiens.CTCF.b1
1,H9-HS.GSE105028.Homo_Sapiens.CTCF.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/H9-HS.GSE105028.Homo_Sapiens.CTCF.b1/hic_results/data/H9-HS.GSE105028.Homo_Sapiens.CTCF.b1/H9-HS.GSE105028.Homo_Sapiens.CTCF.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/H9-HS.GSE105028.Homo_Sapiens.CTCF.b1/MACS2_Ext_No_Control/H9-HS.GSE105028.Homo_Sapiens.CTCF.b1.macs2_peaks.narrowPeak_Q0.01filt,H9-HS.GSE105028.Homo_Sapiens.CTCF.b1
2,H9.GSE105028.Homo_Sapiens.H3K4me1.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/H9.GSE105028.Homo_Sapiens.H3K4me1.b1/hic_results/data/H9.GSE105028.Homo_Sapiens.H3K4me1.b1/H9.GSE105028.Homo_Sapiens.H3K4me1.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/H9.GSE105028.Homo_Sapiens.H3K4me1.b1/MACS2_Ext_No_Control/H9.GSE105028.Homo_Sapiens.H3K4me1.b1.macs2_peaks.narrowPeak_Q0.01filt,H9.GSE105028.Homo_Sapiens.H3K4me1.b1
3,H9-HS.GSE105028.Homo_Sapiens.H3K4me1.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/H9-HS.GSE105028.Homo_Sapiens.H3K4me1.b1/hic_results/data/H9-HS.GSE105028.Homo_Sapiens.H3K4me1.b1/H9-HS.GSE105028.Homo_Sapiens.H3K4me1.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/H9-HS.GSE105028.Homo_Sapiens.H3K4me1.b1/MACS2_Ext_No_Control/H9-HS.GSE105028.Homo_Sapiens.H3K4me1.b1.macs2_peaks.narrowPeak_Q0.01filt,H9-HS.GSE105028.Homo_Sapiens.H3K4me1.b1
4,H9.GSE105028.Homo_Sapiens.KLF4.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/H9.GSE105028.Homo_Sapiens.KLF4.b1/hic_results/data/H9.GSE105028.Homo_Sapiens.KLF4.b1/H9.GSE105028.Homo_Sapiens.KLF4.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/H9.GSE105028.Homo_Sapiens.KLF4.b1/MACS2_Ext_No_Control/H9.GSE105028.Homo_Sapiens.KLF4.b1.macs2_peaks.narrowPeak_Q0.01filt,H9.GSE105028.Homo_Sapiens.KLF4.b1


#### Reviewing the data

In [11]:
print('Unique chipseq peak results: {}'.format(hicpro_to_chipseq.chipseq_std_sample_name.nunique()))

Unique chipseq peak results: 189


In [12]:
# count the number of unique per organim
hicpro_to_chipseq.loc[:, 'organism'] = hicpro_to_chipseq.hicpro_std_sample_name.str.split('.').apply(lambda x: x[2])
org_grps = hicpro_to_chipseq.groupby('organism')
org_grps.apply(lambda df: df.chipseq_std_sample_name.nunique())

organism
GSE212978         2
GSE213385         1
Homo_Sapiens    147
Mus_Musculus     39
dtype: int64

In [13]:
# rogue groups aka, forgot to replace . with -
print('GSE212978')
display(org_grps.get_group('GSE212978'))
print('GSE213385')
display(org_grps.get_group('GSE213385'))

GSE212978


,hicpro_std_sample_name,hicpro_path,chipseq_path,chipseq_std_sample_name,organism
42,JN-DSRCT1.shEWSWT1.GSE212978.Homo_Sapiens.H3K27ac.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/JN-DSRCT1.shEWSWT1.GSE212978.Homo_Sapiens.H3K27ac.b1/hic_results/data/JN-DSRCT1.shEWSWT1.GSE212978.Homo_Sapiens.H3K27ac.b1/JN-DSRCT1.shEWSWT1.GSE212978.Homo_Sapiens.H3K27ac.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/JN-DSRCT1.shEWSWT1.GSE212977.Homo_Sapiens.H3K27ac.b1/MACS2_Ext_No_Control/JN-DSRCT1.shEWSWT1.GSE212977.Homo_Sapiens.H3K27ac.b1.macs2_peaks.narrowPeak_Q0.01filt,JN-DSRCT1.shEWSWT1.GSE212977.Homo_Sapiens.H3K27ac.b1,GSE212978
43,JN-DSRCT1.shGFP.GSE212978.Homo_Sapiens.H3K27ac.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/JN-DSRCT1.shGFP.GSE212978.Homo_Sapiens.H3K27ac.b1/hic_results/data/JN-DSRCT1.shGFP.GSE212978.Homo_Sapiens.H3K27ac.b1/JN-DSRCT1.shGFP.GSE212978.Homo_Sapiens.H3K27ac.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/JN-DSRCT1.shGFP.GSE212977.Homo_Sapiens.H3K27ac.b1/MACS2_Ext_No_Control/JN-DSRCT1.shGFP.GSE212977.Homo_Sapiens.H3K27ac.b1.macs2_peaks.narrowPeak_Q0.01filt,JN-DSRCT1.shGFP.GSE212977.Homo_Sapiens.H3K27ac.b1,GSE212978


GSE213385


,hicpro_std_sample_name,hicpro_path,chipseq_path,chipseq_std_sample_name,organism
44,L3.6pl.GSE213385.Homo_Sapiens.H3K27ac.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/L3.6pl.GSE213385.Homo_Sapiens.H3K27ac.b1/hic_results/data/L3.6pl.GSE213385.Homo_Sapiens.H3K27ac.b1/L3.6pl.GSE213385.Homo_Sapiens.H3K27ac.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/L3.6pl.GSE213378.Homo_Sapiens.H3K27ac.b1/MACS2_Ext_No_Control/L3.6pl.GSE213378.Homo_Sapiens.H3K27ac.b1.macs2_peaks.narrowPeak_Q0.01filt,L3.6pl.GSE213378.Homo_Sapiens.H3K27ac.b1,GSE213385
45,L3.6pl.GSE213385.Homo_Sapiens.H3K27ac.b2,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/L3.6pl.GSE213385.Homo_Sapiens.H3K27ac.b2/hic_results/data/L3.6pl.GSE213385.Homo_Sapiens.H3K27ac.b2/L3.6pl.GSE213385.Homo_Sapiens.H3K27ac.b2.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/L3.6pl.GSE213378.Homo_Sapiens.H3K27ac.b1/MACS2_Ext_No_Control/L3.6pl.GSE213378.Homo_Sapiens.H3K27ac.b1.macs2_peaks.narrowPeak_Q0.01filt,L3.6pl.GSE213378.Homo_Sapiens.H3K27ac.b1,GSE213385


#### Generate the main ChIP-seq dataframe 

In [14]:
org = "Homo"

In [15]:
# generate the chipseq dataframe, update rerun as needed
outfn = os.path.join(outdir, f'{ref}.analysis.peaks.summary.tsv')
chipseq_df = generate_chipseq_df(outfn=outfn, rerun=rerun)

# add column  names and sort
chipseq_df.columns = ["sample_name", "num_peaks_chipseq", "avg_peak_size_chipseq"]
chipseq_df = chipseq_df.sort_values(by=["sample_name"], ascending=True).reset_index(drop=True)

# add chipseq std sample name and remove duplicates
chipseq_df = chipseq_df.merge(hicpro_to_chipseq, left_on='sample_name', right_on='hicpro_std_sample_name')
chipseq_df.drop_duplicates(subset=['chipseq_std_sample_name'], inplace=True)

In [16]:
chipseq_df.head()

,sample_name,num_peaks_chipseq,avg_peak_size_chipseq,hicpro_std_sample_name,hicpro_path,chipseq_path,chipseq_std_sample_name,organism
0,293T-PDS.GSE128106.Homo_Sapiens.YY1.b1,257007,589.534522,293T-PDS.GSE128106.Homo_Sapiens.YY1.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/293T-PDS.GSE128106.Homo_Sapiens.YY1.b1/hic_results/data/293T-PDS.GSE128106.Homo_Sapiens.YY1.b1/293T-PDS.GSE128106.Homo_Sapiens.YY1.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/293T-PDS.GSE128106.Homo_Sapiens.YY1.b1/MACS2_Ext_No_Control/293T-PDS.GSE128106.Homo_Sapiens.YY1.b1.macs2_peaks.narrowPeak_Q0.01filt,293T-PDS.GSE128106.Homo_Sapiens.YY1.b1,Homo_Sapiens
1,293T-TMPYP4.GSE128106.Homo_Sapiens.YY1.b1,238269,566.071726,293T-TMPYP4.GSE128106.Homo_Sapiens.YY1.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/293T-TMPYP4.GSE128106.Homo_Sapiens.YY1.b1/hic_results/data/293T-TMPYP4.GSE128106.Homo_Sapiens.YY1.b1/293T-TMPYP4.GSE128106.Homo_Sapiens.YY1.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/293T-TMPYP4.GSE128106.Homo_Sapiens.YY1.b1/MACS2_Ext_No_Control/293T-TMPYP4.GSE128106.Homo_Sapiens.YY1.b1.macs2_peaks.narrowPeak_Q0.01filt,293T-TMPYP4.GSE128106.Homo_Sapiens.YY1.b1,Homo_Sapiens
2,293T.GSE128106.Homo_Sapiens.YY1.b1,259289,549.176625,293T.GSE128106.Homo_Sapiens.YY1.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/293T.GSE128106.Homo_Sapiens.YY1.b1/hic_results/data/293T.GSE128106.Homo_Sapiens.YY1.b1/293T.GSE128106.Homo_Sapiens.YY1.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/293T.GSE128106.Homo_Sapiens.YY1.b1/MACS2_Ext_No_Control/293T.GSE128106.Homo_Sapiens.YY1.b1.macs2_peaks.narrowPeak_Q0.01filt,293T.GSE128106.Homo_Sapiens.YY1.b1,Homo_Sapiens
3,A673-siCT-Dh1-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1,115154,630.902669,A673-siCT-Dh1-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/A673-siCT-Dh1-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1/hic_results/data/A673-siCT-Dh1-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1/A673-siCT-Dh1-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/A673-siCT-Dh1-72h-D347-D362.GSE156649.Homo_Sapiens.CTCF.b1/MACS2_Ext_No_Control/A673-siCT-Dh1-72h-D347-D362.GSE156649.Homo_Sapiens.CTCF.b1.macs2_peaks.narrowPeak_Q0.01filt,A673-siCT-Dh1-72h-D347-D362.GSE156649.Homo_Sapiens.CTCF.b1,Homo_Sapiens
5,A673-siSA2-Dh6-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1,102935,615.880789,A673-siSA2-Dh6-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/A673-siSA2-Dh6-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1/hic_results/data/A673-siSA2-Dh6-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1/A673-siSA2-Dh6-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline_v2/A673-siSA2-Dh6-72h-D347-D362.GSE156649.Homo_Sapiens.CTCF.b1/MACS2_Ext_No_Control/A673-siSA2-Dh6-72h-D347-D362.GSE156649.Homo_Sapiens.CTCF.b1.macs2_peaks.narrowPeak_Q0.01filt,A673-siSA2-Dh6-72h-D347-D362.GSE156649.Homo_Sapiens.CTCF.b1,Homo_Sapiens


In [17]:
chipseq_df.shape

(150, 8)

In [18]:
#has_chipseq = df.loc[df['num_peaks_fithichip'] == 0]
len(chipseq_df.loc[chipseq_df["num_peaks_chipseq"] != -1])
#print(len(has_chipseq))
#has_inferred = has_chipseq.loc[df['num_peaks_fithichip'] != 0]
#low = ((has_inferred.num_peaks_hichip_peaks < 5000) | (has_inferred.num_peaks_fithichip < 5000) | (has_inferred.num_peaks_chipseq < 5000))
#has_inferred[low].drop(columns=['avg_peak_size_hichip_peaks', 'avg_peak_size_fithichip', 'avg_peak_size_chipseq']).reset_index(drop=True)

print('Problem with these samples here:')
display(chipseq_df.loc[chipseq_df["num_peaks_chipseq"] == -1])

Problem with these samples here:


,sample_name,num_peaks_chipseq,avg_peak_size_chipseq,hicpro_std_sample_name,hicpro_path,chipseq_path,chipseq_std_sample_name,organism
32,CTL.GSE117888.Homo_Sapiens.H3K27ac.b1,-1,-1.0,CTL.GSE117888.Homo_Sapiens.H3K27ac.b1,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/hicpro/CTL.GSE117888.Homo_Sapiens.H3K27ac.b1/hic_results/data/CTL.GSE117888.Homo_Sapiens.H3K27ac.b1/CTL.GSE117888.Homo_Sapiens.H3K27ac.b1.allValidPairs,/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/Could not find a matching file,-1,Homo_Sapiens


In [19]:
# save the summary for the loop catalog website/later use
save_df = chipseq_df[['chipseq_std_sample_name', 'num_peaks_chipseq', 'avg_peak_size_chipseq']]

# save in excel format
outfn = f"results/tables/final.all_batches/{ref}.unmerged_peaks.chipseq.{assigned_date}.xlsx"
save_df.to_excel(outfn); os.chmod(outfn, 0o664)

# save in tsv format
outfn = f"results/tables/final.all_batches/{ref}.unmerged_peaks.chipseq.{assigned_date}.tsv"
save_df.to_csv(outfn, sep = "\t", index = None); os.chmod(outfn, 0o664)

# Load FithiChIP Data

In [20]:
def read_peak_files(file):
    peak_data = {}
    count = 0
    with open(file) as fr:
        for line in fr:
            info = line.strip().split()
            peak_info = info[0:3]
            peak_data[count] = peak_info
            count = count + 1
    return(peak_data)

In [21]:
def generate_fithichip_df(outfn='test.tsv', rerun=False):
    
    if rerun == True:
        data = []
        glob_str = 'results/hicpro/*Homo*/hic_results'

        for i, file in enumerate(glob.glob(glob_str)):
            
            # get sample information
            sample_name = file.split('/')[2]
            sample_info = []
            sample_info.append(sample_name)

            ## FitHiChIP peaks
            fithichip_path = 'results/peaks/fithichip/{sn}/MACS2_ExtSize/out_macs2_peaks.narrowPeak'.format(sn = sample_name)
            if os.path.exists(fithichip_path):
                
                # count the number of peaks
                peak_info = read_peak_files(fithichip_path)
                sample_info.append(len(peak_info))
                
                # averaging peak sizes
                peak_sizes = []
                for peak in peak_info.items():
                    peak_sizes.append(int(int(peak[1][2])-int(peak[1][1])))
                sample_info.append(np.mean(peak_sizes))

            else:
                
                # setting defaults/not present values, -1 is used to signify this
                sample_info.append(-1)
                sample_info.append(-1)

            data.append(sample_info)
                
            if i % 100 == 0:
                print('Processed: {}'.format(i))

        # create a dataframe and save to file
        df = pd.DataFrame(data)
        df.to_csv(outfn, sep='\t', index=None)
        
    else:
        # load old data when re-run is not required
        df = pd.read_table(outfn)
        
    return df

### Loading data

#### Generate the main fithichip dataframe 

In [22]:
org = "Homo"

In [23]:
# generate the chipseq dataframe, update rerun as needed
outfn = os.path.join(outdir, f'{ref}.analysis.peaks.summary.fithichip.tsv')
fithichip_df = generate_fithichip_df(outfn=outfn, rerun=rerun)

Processed: 0


/mnt/BioAdHoc/Groups/vd-ay/jreyna/software/Herman-Cluster-v3/mambaforge-pypy3/envs/lc-pipelines/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/BioAdHoc/Groups/vd-ay/jreyna/software/Herman-Cluster-v3/mambaforge-pypy3/envs/lc-pipelines/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Processed: 100
Processed: 200
Processed: 300
Processed: 400
Processed: 500
Processed: 600
Processed: 700


In [24]:
# add column  names and sort
fithichip_df.columns = ["sample_name", "num_peaks_fithichip", "avg_peak_size_fithichip"]
fithichip_df = fithichip_df.sort_values(by=["sample_name"], ascending=True).reset_index(drop=True)

In [25]:
fithichip_df.head()

,sample_name,num_peaks_fithichip,avg_peak_size_fithichip
0,293T-PDS.GSE128106.Homo_Sapiens.YY1.b1,29430,174.444852
1,293T-TMPYP4.GSE128106.Homo_Sapiens.YY1.b1,73184,187.952298
2,293T.GSE128106.Homo_Sapiens.YY1.b1,987,183.697062
3,A375.GSE210423.Homo_Sapiens.H3K27ac.b1,45783,476.322172
4,A673-siCT-Dh1-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1,11489,265.430151


In [26]:
fithichip_df.shape

(763, 3)

In [27]:
#has_chipseq = df.loc[df['num_peaks_fithichip'] == 0]
len(fithichip_df.loc[fithichip_df["num_peaks_fithichip"] != -1])
#print(len(has_chipseq))
#has_inferred = has_chipseq.loc[df['num_peaks_fithichip'] != 0]
#low = ((has_inferred.num_peaks_hichip_peaks < 5000) | (has_inferred.num_peaks_fithichip < 5000) | (has_inferred.num_peaks_chipseq < 5000))
#has_inferred[low].drop(columns=['avg_peak_size_hichip_peaks', 'avg_peak_size_fithichip', 'avg_peak_size_chipseq']).reset_index(drop=True)

print('Problem with these samples here:')
display(fithichip_df.loc[fithichip_df["num_peaks_fithichip"] == -1])

Problem with these samples here:


,sample_name,num_peaks_fithichip,avg_peak_size_fithichip
591,T47D-T0.GSE179666.Homo_Sapiens.PGR.b1,-1,-1.0
592,T47D-T30.GSE179666.Homo_Sapiens.PGR.b1,-1,-1.0
650,VCaP-AU-4h.GSE171591.Homo_Sapiens.CTCF.b1,-1,-1.0
651,VCaP-AU-4h.GSE171591.Homo_Sapiens.H3K27ac.b1,-1,-1.0
652,VCaP-AU-4h.GSE171591.Homo_Sapiens.H3K4me3.b1,-1,-1.0
653,VCaP-DMSO-4h.GSE171591.Homo_Sapiens.CTCF.b1,-1,-1.0
654,VCaP-DMSO-4h.GSE171591.Homo_Sapiens.H3K27ac.b1,-1,-1.0
655,VCaP-DMSO-4h.GSE171591.Homo_Sapiens.H3K4me3.b1,-1,-1.0


In [28]:
# save the summary for the loop catalog website/later use
save_df = fithichip_df[['sample_name', 'num_peaks_fithichip', 'avg_peak_size_fithichip']]

# save in excel format
outfn = f"results/tables/final.all_batches/{ref}.unmerged_peaks.fithichip.{assigned_date}.xlsx"
save_df.to_excel(outfn); os.chmod(outfn, 0o664)

# save in tsv format
outfn = f"results/tables/final.all_batches/{ref}.unmerged_peaks.fithichip.{assigned_date}.tsv"
save_df.to_csv(outfn, sep = "\t", index = None); os.chmod(outfn, 0o664)

In [29]:
7 + 7

14